### Start with importing the packages needed.

In [64]:
import logging
import json
import textwrap
import re

from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.filters import FunctionInvocationContext
from semantic_kernel.functions import KernelFunctionFromPrompt, KernelArguments
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.agents.strategies import KernelFunctionTerminationStrategy, KernelFunctionSelectionStrategy
from pydantic import BaseModel, Field
from typing import List
from IPython.display import display, Markdown

# Load environment variables from .env file
load_dotenv()

# NOTE: This is all that is required to enable logging.
# Set the desired level to INFO, DEBUG, etc.
logging.basicConfig(level=logging.WARNING)

In [ ]:
class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class Article(BaseModel):
    title: str
    content: str
    call_to_action: str

class ContentOutput(BaseModel):
    article: Article
    social_media_posts: List[SocialMediaPost]

In [ ]:
# Some casual things for formatting.

intermediate_steps: list[ChatMessageContent] = []

async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    intermediate_steps.append(message)

output_settings = AzureChatPromptExecutionSettings(response_format=ContentOutput)

In [ ]:
# Create and configure the kernel.
from search import SearchPlugin


service_id: str = "default_service_id"
kernel = Kernel(service_id=service_id)

kernel.add_plugin(SearchPlugin(), plugin_name="search")
kernel.add_service(AzureChatCompletion(service_id=service_id, deployment_name="gpt-4o"))

In [ ]:
# Create the Lead Market Analyst agent. NOTE: The name of the agent must not contain spaces.
lead_research_agent = ChatCompletionAgent(
    service= AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="LeadResearchAgent",
    instructions=(
    """
    As the Lead Research Analyst at a premier content platform, you specialize in finding information that best fits the needs of your
    audience. Your primary responsibility is to ensure that the content produced by your team is not only accurate but also timely and
    relevant. You will do this by conducting in-depth research. You will need to search the web to get this content.
    """),
    plugins=[
        SearchPlugin()
    ],
    
)

In [ ]:
# Create the Data Analyst agent. NOTE: The name of the agent must not contain spaces.
data_analyst_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="DataAnalystAgent",
    instructions=(
    """
    As the Chief Data Strategist at a leading advisory firm, your expertise lies in analyzing vast datasets to uncover trends and
    opportunities that inform high-impact strategies with the goal of Synthesizing complex data into actionable insights 
    that can be transformed into compelling content.
    """),
)

In [ ]:
# Create the Content Creator agent. NOTE: The name of the agent must not contain spaces.
blog_creator_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="BlogCreatorAgent",
    instructions=(
    """
    As the Content Creator at a leading advisory firm, your role is to transform complex data and insights into
    engaging and informative content that resonates with our audience. You will be responsible for creating articles, blogs, and other
    materials that effectively communicate our findings and recommendations. Additionally, you will collaborate with other agents to ensure
    the content aligns with our overall strategy and objectives.
    """),
)

In [ ]:
social_creator_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="SocialCreatorAgent",
    instructions=(
    """
    As the Social Media Manager at a leading advisory firm, your role is to create engaging and impactful content for our social media
    platforms. You will be responsible for crafting tweets, posts, and other materials that effectively communicate our findings and
    recommendations. Any twitter posts, Instagram posts, and other posts should be very short and easy to understand, and should sound like clickbait.
    Additionally, you will collaborate with other agents to ensure the content aligns with our overall strategy and objectives.
    """),
)

In [66]:
# Create the Quality Assurance agent. NOTE: The name of the agent must not contain spaces.
quality_assurance_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="QualityAssuranceAgent",
    arguments=KernelArguments(settings=output_settings),
    instructions=(
    """
    As the Quality Assurance Specialist at a leading advisory firm, your role is to ensure that all content produced meets our
    high standards for accuracy, clarity, and relevance. You will review and edit content created by the team to ensure it aligns with our
    brand voice and effectively communicates our insights. Additionally, you will provide feedback to the social_creator_agent and the blog_creator_agent to enhance content quality.
    If the content is not good enough, you will need to explain why it is not good enough and what needs to be changed.
    If it is good enough, you will need to say that it is good enough and that it can be published.
    """),
)

In [ ]:
# Create the Coordinator agent. NOTE:The name of the agent must not contain spaces.
coordinator_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    name="CoordinatorAgent",
    instructions=(
    """
    As the Coordinator at a leading advisory firm, your role is to facilitate communication and collaboration among team members,
    so that they can do research, write a blog, and create social media posts. You will be responsible for
    coordinating the efforts of the Lead Research Analyst, Data Analyst, Blog Creator, and Social Media Manager to ensure that all
    content is aligned with our overall strategy and objectives.
    
    """),
    plugins=[ ],
)

In [ ]:
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt="""
    Based on the conversation history, determine the next agent to take a turn. 
    Only return the name of the agent from the following list:
    - LeadResearchAgent
    - DataAnalystAgent
    - BlogCreatorAgent
    - SocialCreatorAgent
    - QualityAssuranceAgent

    Rules:
    - LeadResearchAgent starts first.
    - After LeadResearchAgent, DataAnalystAgent speaks.
    - After DataAnalystAgent, BlogCreatorAgent speaks.
    - After BlogCreatorAgent speaks, the QualityAssuranceAgent speaks.
    - If QualityAssuranceAgent requests changes, restart the cycle with BlogCreatorAgent.
    - If the QualityAssuranceAgent approves the content, the SocialCreatorAgent speaks.
    - After SocialCreatorAgent speaks, the QualityAssuranceAgent speaks.
    - If the QualityAssuranceAgent requests changes, restart the cycle with SocialCreatorAgent.
    - If the QualityAssuranceAgent approves the content, the cycle ends.

    History:
    {{$history}}
    """,
)

In [60]:
termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt="""
    Determine if the structured content output (article and social_media_posts) is complete and valid JSON.
    If the output is complete and valid, respond with "yes". Otherwise, respond with "no".
    Ensure the JSON adheres to the schema provided.

    History:
    {{$history}}
    """
)

In [67]:
agent_group = AgentGroupChat(
    agents=[lead_research_agent, data_analyst_agent, blog_creator_agent, social_creator_agent, quality_assurance_agent, coordinator_agent],
    termination_strategy = KernelFunctionTerminationStrategy(
        agents=[quality_assurance_agent],
        function=termination_function,
        kernel=kernel,
        result_parser=lambda result: any("yes" in item.content.lower() for item in result.value) if isinstance(result.value, list) else "yes" in result.value.content.lower(),
        history_variable_name="history",
        maximum_iterations=10
    ),
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=kernel,
        result_parser=lambda result: result.value[0].content.strip() if isinstance(result.value, list) and result.value else result.value.content.strip(),
        agent_variable_name="agents",
        history_variable_name="history"
    )
)

In [68]:
thread: ChatHistoryAgentThread = None


topic = "Inflation in the US and the impact on the stock market in 2025"
initial_message = f"Subject: {topic}\n\nCollaborate to produce structured content output matching the provided schema."
await agent_group.add_chat_message(message=initial_message)

final_response = ""
async for response in agent_group.invoke():
    print(f"# {response.name}: {response.content}\n")
    final_response = response.content

print (f"Final Response: {response.content}")

# Print the intermediate steps
#print("\nIntermediate Steps:")
#for msg in intermediate_steps:
#    if any(isinstance(item, FunctionResultContent) for item in msg.items):
#        for fr in msg.items:
#            if isinstance(fr, FunctionResultContent):
#                print(f"\033[33mFunction Result for function: {fr.name} > \033[0m{fr.result} ")
#    elif any(isinstance(item, FunctionCallContent) for item in msg.items):
#        for fcc in msg.items:
#            if isinstance(fcc, FunctionCallContent):
#                print(f"\033[33mFunction Call:> {fcc.name} with arguments: {fcc.arguments}\033[0m")
#    else:
#        print(f"\033[34m{msg.role}: {msg.content}\033[0m")

Performing search...
Query: Inflation in the US and stock market impact predictions for 2025
Performing search...
Query: US inflation trends and stock market outlook 2025
# LeadResearchAgent: Here's a structured summary on the topic "Inflation in the US and the Impact on the Stock Market in 2025":

---

### **1. Current Inflation Trends in the US Leading into 2025**
- Inflation remains a critical concern leading into 2025. Factors like geopolitical risks, persisting supply chain disruptions, and evolving monetary policies have substantially influenced inflation rates.
- A sticky inflation outlook suggests that price levels may not normalize quickly, and the Federal Reserve might choose a more cautious approach to interest rate adjustments.
- Consumer Price Index (CPI), utilized as a primary gauge of inflation, indicates mixed signals. While recent data shows incremental improvements, inflation levels remain above historical averages heading into 2025.

**Source Highlights:**
- [Morning

In [65]:
def extract_json(text: str) -> dict:
    pattern = re.compile(r"```json\s*(\{.*\})\s*```", re.DOTALL)
    match = pattern.search(text)
    if match:
        return json.loads(match.group(1))
    return json.loads(text.strip())

output_data = extract_json(final_response)
article = output_data.get("article", {})
posts = output_data.get("social_media_posts", [])

article_md = f"# {article.get('title')}\n\n{article.get('content')}\n\n**{article.get('call_to_action')}**"
display(Markdown(article_md))

for post in posts:
    print(f"Platform: {post['platform']}")
    print(textwrap.fill(post['content'], width=60))
    print('-' * 60)

# Navigating Inflation Impact on the US Stock Market in 2025

### Understanding Inflation Trends in 2025

Inflation in the United States during 2025 has proven to be a persistent economic challenge, influencing monetary policies and market dynamics. Here, we outline the key factors behind current inflation trends and their significant effects on the US stock market.

### Key Drivers of Inflation in 2025

- **Energy Prices**: Energy costs are influencing a noticeable rise in consumer price indices, impacting operating costs across industries.
- **Global Trade Policies**: Import tariffs and international trade restrictions remain critical inflationary pressures.

### Stock Market Dynamics Amid Inflation

- **Interest Rate Adjustments**: The Federal Reserve's approach to managing inflation involves maintaining higher-than-average interest rates, which has direct implications for investment yields and borrowing costs in the financial markets.
- **Sectoral Impacts**: Sectors such as technology and financial services show increased volatility, while defensive stocks in utilities and healthcare provide relative stability.

### Strategic Investment Approaches

1. **Sectoral Shifts**: Favor inflation-resistant sectors such as energy and utilities.
2. **Diversification**: Expand investments across different asset classes to mitigate risks associated with specific sectors.
3. **Long-term Holdings**: Focus on securities showing resilience in turbulent economic conditions.

Staying informed about inflationary trends and adopting meticulous financial strategies can help mitigate risks and leverage high inflation periods' opportunities.

**For more in-depth insights on navigating economic fluctuations and strengthening investment portfolios in 2025, consult our financial analysis team.**

Platform: LinkedIn
Inflation in the US during 2025 continues to impact stock
market dynamics. Learn how sectors like utilities and
healthcare provide stability in turbulent times. Stay ahead
with our comprehensive analysis. #FinancialAnalysis
#Inflation2025 #InvestmentStrategies
------------------------------------------------------------
Platform: Twitter
How is inflation shaping the US stock market in 2025? 📈
Explore trends and investment strategies for benchmark
stability. #Inflation #StockMarket #InvestmentTrends
------------------------------------------------------------
Platform: Facebook
Inflation reshapes the 2025 investment landscape. Discover
key trends in the US economy and how your portfolio can
weather inflationary pressures. Visit our website for expert
recommendations! #FinanceInsights #InvestSmart
------------------------------------------------------------
